<a href="https://colab.research.google.com/github/kingbingodbin/NLP_models/blob/main/XLnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as data_utils
from torchtext.data.utils import get_tokenizer
from torchtext.data import Field
import pandas as pd
import os
import zipfile
from itertools import combinations

In [12]:
import torchtext
import io

In [13]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [14]:
import os
os.listdir('drive/My Drive/datasets/wikitext-2')

['wiki.valid.tokens', 'wiki.test.tokens', 'wiki.train.tokens']

In [15]:
#install package for subword tokenizer
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 5.5MB/s 


In [16]:
#install package for subword tokenizer
!pip install revtok

tokenizer

In [ ]:
import sentencepiece as spm

In [ ]:
import urllib.request
import io
import sentencepiece as spm

# Loads model from URL as iterator and stores the model to BytesIO.
model = io.BytesIO()
with urllib.request.urlopen(
    'https://raw.githubusercontent.com/google/sentencepiece/master/data/botchan.txt'
) as response:
  spm.SentencePieceTrainer.train(
      sentence_iterator=response, model_writer=model, vocab_size=1000)

# Serialize the model as file.
# with open('out.model', 'wb') as f:
#   f.write(model.getvalue())

# Directly load the model from serialized model.
sp = spm.SentencePieceProcessor(model_proto=model.getvalue())
print(sp.encode('this is test'))

[64, 47, 4, 13, 391]


In [ ]:
aa = torchtext.datasets.language_modeling.WikiText2.splits()

TypeError: ignored

In [ ]:
"""load wikitext2 data"""

In [178]:
tokenizer = get_tokenizer('revtok')

In [185]:
from revtok import tokenizer

AttributeError: ignored

In [274]:
dataset_base_path = 'drive/My Drive/datasets/'
path = dataset_base_path + 'wikitext-2/'
#use reversable tokenizer for flexible 
TEXT = torchtext.data.ReversibleField(tokenize = get_tokenizer("revtok"),
                            init_token = '<sos>',
                            eos_token = '<eos>',
                            unk_token ='<unk>',
                            sequential=True,
                            lower=True,
                            use_vocab=True)

In [275]:
dtype = torch.float
device = torch.device("cpu")

In [276]:
tr_dat,val_dat,te_dat = torchtext.datasets.language_modeling.WikiText2.splits(TEXT)

In [277]:
[tr_dat.examples[0].text][0][:10]

[' ',
 ' ',
 '<eos>',
 ' ',
 ' = ',
 ' valkyria ',
 ' chronicles ',
 ' iii ',
 ' = ',
 ' ']

In [278]:
[val_dat.examples[0].text][0][:10]

[' ', ' ', '<eos>', ' ', ' = ', ' homarus ', ' gammarus ', ' = ', ' ', '<eos>']

In [279]:
[te_dat.examples[0].text][0][:10]

[' ', ' ', '<eos>', ' ', ' = ', ' robert ', ' <', ' unk ', '> ', ' = ']

In [280]:
hasattr(TEXT, "vocab")

False

In [281]:
#seperating wikitext2 data to documents
def sep_to_docs(data : list, doc_sep : str, token_sep : str) -> list:
  """returns documents from split input text chunk
  """
  result = []
  docs = (''.join(data)).split(doc_sep)
  for i in docs:
    curr_doc = (''.join(i)).split(token_sep)
    curr_doc = [i for i in curr_doc if i != '']
    if len(curr_doc) >= 1:
      result.append(curr_doc)
  return result

In [282]:
tr_docs = sep_to_docs(data = [tr_dat.examples[0].text][0], doc_sep = '<eos>  =', token_sep = '  ')
val_docs = sep_to_docs(data = [val_dat.examples[0].text][0], doc_sep = '<eos>  =', token_sep = '  ')
te_docs = sep_to_docs(data = [te_dat.examples[0].text][0], doc_sep = '<eos>  =', token_sep = '  ')

In [283]:
print(len(tr_docs),len(val_docs),len(te_docs))

6211 620 708


In [258]:
#create vocab with glove.6B.100d
#TEXT.build_vocab(tr_docs,min_freq=10,vectors="glove.6B.100d")
#!cp .vector_cache/glove.6B.zip drive/"My Drive"/datasets/


#TEXT.build_vocab(tr_docs,min_freq=10)

In [211]:
"""extracting vocab"""
#TEXT.build_vocab(tr_docs,min_freq=10,vectors="glove.6B.100d")
#!cp .vector_cache/glove.6B.zip drive/"My Drive"/datasets/
#glove_zip = zipfile.ZipFile('drive/My Drive/datasets/glove.6B.zip')
#glove_zip.extractall('drive/My Drive/datasets/pretrained_wordvectors')
#glove_zip.close()

'extracting vocab'

In [284]:
from torchtext.vocab import Vectors
glove_6b_300d_vec = Vectors(name='glove.6B.300d.txt', cache='drive/My Drive/datasets/pretrained_wordvectors')

special_tokens = '<cls>'
TEXT.build_vocab(tr_docs, min_freq=10, vectors=glove_6b_300d_vec)

torchtext.vocab.Vocab

In [262]:
#<cls>, <sep> 추가

TEXT.build_vocab([['<cls>','<sep>']])

In [267]:
TEXT.vocab.vectors.shape

torch.Size([12983, 300])

In [269]:
aa = torchtext.data.Field()
aa.build_vocab([['<cls>','<sep>']])

In [270]:
TEXT.vocab.extend(aa.vocab)

In [341]:
TEXT.build_vocab(tr_docs, min_freq=10, specials=['<sep>', '<cls>'])

TypeError: ignored

In [ ]:
vocab_size = len(TEXT.vocab.itos)
word_embedding_size = TEXT.vocab.vectors.shape[1]

In [ ]:
#sample embedding
embeds = nn.Embedding(vocab_size, word_embedding_size)
lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
hello_embed = embeds(lookup_tensor)
print(hello_embed)

NameError: ignored

In [136]:
a=[]
a.append([1,2,3])
a

[[1, 2, 3]]

In [ ]:
TEXT.batch_first=True

In [10]:
for i in combinations([1,2,3],2):
  if self.doc_index[i[0-1]:i[0]] != self.doc_index[i[0]]:
    
  else :
    (aa[i[0-1]:i[0]],'<cls>',aa[i[0]],'<sep>',aa[i[1]])



[(1, 2), (1, 3), (2, 3)]

In [293]:
[1,2,3] +[[1]]

[1, 2, 3, [1]]

In [ ]:
torch.tensor(list(range(len([1,2,3,4]))))

tensor([0, 1, 2, 3])

In [388]:
def build_Field_with_vocab(text_dat: list, Field: torchtext.data.Field, special_tokens: list=None, min_freq: int=10, learned_vectors: Vectors=None):

  if special_tokens is not None:
    text_dat = text_dat + [special_tokens * min_freq]

  if isinstance(learned_vectors, Vectors):
    Field.build_vocab(text_dat, min_freq=min_freq, vectors=learned_vectors)
  else:
    Field.build_vocab(text_dat, min_freq=min_freq)
  
  return Field


In [391]:
TEXT = build_Field_with_vocab(text_dat=tr_docs, Field=TEXT, special_tokens =['<cls>', '<sep>'], min_freq=10, learned_vectors=glove_6b_300d_vec)

6211
[['=', '=', 'control', '=', '=', '=', '<eos>', '<eos>', 'because', 'of', 'the', 'damage', 'they', 'do', ',', 'there', 'have', 'been', 'attempts', 'to', 'control', 'the', 'numbers', 'of', 'both', 'native', 'and', 'introduced', 'populations', 'of', 'common', 'starlings', '.', 'within', 'the', 'natural', 'breeding', 'range', ',', 'this', 'may', 'be', 'affected', 'by', 'legislation', '.', 'for', 'example', ',', 'in', 'spain', ',', 'this', 'is', 'a', 'species', 'hunted', 'commercially', 'as', 'a', 'food', 'item', ',', 'and', 'has', 'a', 'close', 'season', ',', 'whereas', 'in', 'france', ',', 'it', 'is', 'classed', 'as', 'a', 'pest', ',', 'and', 'the', 'season', 'in', 'which', 'it', 'may', 'be', 'killed', 'covers', 'the', 'greater', 'part', 'of', 'the', 'year', '.', 'in', 'the', 'uk', ',', 'the', 'common', 'starling', 'may', 'be', 'killed', 'at', 'any', 'time', 'of', 'year', '.', 'this', 'species', 'is', 'migratory', ',', 'so', 'birds', 'involved', 'in', 'control', 'measures', 'may', 'h

In [291]:
class Xlnet_Input(torch.utils.data.Dataset):
  """Creates input data with unlabeled text data
   input : list of unlabeled text in list form
   output : dataset, list of tuple(Text, seq_len, context, relative_position, segment)"""
  def __init__(self, text_dat: list, Field: Field, sep_docs: bool=True, doc_sep: str='<eos>  =', token_sep : str='  '):
    """text_dat: list of unlabeled documents(list of lists), should be tokenized with 'revtok'
    """
    #dict
    #context_encoding, segment_encoding
    #positional encoding

    self.doc_sep = doc_sep
    self.token_sep = token_sep

    if sep_docs:
      text_dat = self.sep_to_docs(text_dat)
    
    if special_tokens is not None:
      text_dat.append(special_tokens)

    if build_dict:
      TEXT.build_vocab(tr_docs, min_freq=10, vectors=)
      

    
    self.text_dat =[]
    for doc in text_dat:
      self.text_dat.append(self.Field.numericalize([doc]))
    
    self.text_dat = []
    self.seq_len = []
    self.context = []
    self.relative_position = []
    self.segment = []
    self.input = []

  def create_xlnet_input(self, segment_length: int, reuse_length: int, seq_length: int):
    """creates input data sample for xlnet,
    create all dataset and allocate memory for those dataset.
    this will reduce time for __getitem__ operation"""
    #self.segment_len = segment_len
    self.seq_length = seq_length
    self.reuse_length = reuse_length


    self.segmentation(self, segment_len)


    self.text_segment = []
    self.position_segment = []
    self.segment_index = []
    self.doc_index = []
    
    self.input = []

    del self.input_text
    del self.text_len
    del self.context
    del self.relative_position
    del self.segment
    
  def segmentation(self, segment_len):
    """split input text into segments
    inputs 
      segment_len : length to which input text is splitted
    
    creates
      text_segment : segmented text data
      position_segment : positional information of text segments
      segment_index : index of segment
      doc_index : index of the documnet(or context)
    """
    self.text_segment = []
    self.position_segment = []
    self.segment_index = []
    self.doc_index = []
    
    curr_doc_index = 0

    if()

    for curr_doc in self.input_text:

      self.segment_index.append(list(range(len(curr_doc) // self.segment_len + 1)))

      self.doc_index.append([curr_doc_index] * len(self.segment_index))

      curr_doc_index += 1

      for curr_index in self.segment_index[-1]:
        
        self.text_segment.append(self.text_dat[self.segment_len * curr_index: (self.segment_len * curr_index + 1)])
        
        last_segment_len = len(text_segment[-1])
        
        self.position_segment.append( list( range( self.segment_len * curr_index, self.segment_len * curr_index + last_segment_len)))


        



      
    
  def sep_to_docs(self) -> list:
  #def sep_to_docs(data : list, doc_sep : str, token_sep : str) -> list:
  """returns documents from split input text chunk
  """
    result = []
    docs = (''.join(data)).split(doc_sep)
    for i in docs:
      curr_doc = (''.join(i)).split(token_sep)
      curr_doc = [i for i in curr_doc if i != '']
      if len(curr_doc) >= 1:
        result.append(curr_doc)
    return result

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()

    doc_paths = os.path.join(self.root_dir, self.target_csv.iloc[idx, 0])
    for doc_path in doc_paths
    doc_file = open(doc_path, 'r')

    lines = doc_file.readlines()

    if word_dict:
      for line in lines:
        for 

    


    doc_file.close()


    f = open("C:/doit/새파일.txt", 'r')


      landmarks = self.landmarks_frame.iloc[idx, 1:]
      landmarks = np.array([landmarks])
      landmarks = landmarks.astype('float').reshape(-1, 2)
      sample = {'image': image, 'landmarks': landmarks}

    return sample

  def __len__(self):
    return len(self.input)

SyntaxError: ignored

In [ ]:
self.target_csv = pd.read_csv(traget_csv)
    self.dat_shape = data.shape
    self.root_dir = root_dir
    self.segment_len = segment_len
    self.word_dict = False
    self.data = data

    if do_shuffle:
      shuffle()
    
    segmentation()

  def segmentation(self):
    self.segments = 

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.tolist()

    doc_paths = os.path.join(self.root_dir, self.target_csv.iloc[idx, 0])
    for doc_path in doc_paths
    doc_file = open(doc_path, 'r')

    lines = doc_file.readlines()

    if word_dict:
      for line in lines:
        for 

    


    doc_file.close()


    f = open("C:/doit/새파일.txt", 'r')


      landmarks = self.landmarks_frame.iloc[idx, 1:]
      landmarks = np.array([landmarks])
      landmarks = landmarks.astype('float').reshape(-1, 2)
      sample = {'image': image, 'landmarks': landmarks}

    return sample

  def __len__(self):
    return len(self.d)

'<eos>  ='

In [ ]:
aa = [tr_dat.examples[0].text][0][:1000]

In [ ]:
result = []
docs = (''.join(aa)).split('<eos>  =')
for i in temp:
  curr_doc = (''.join(i)).split('  ')
  curr_doc = [i for i in curr_doc if i != '']
  result.append(curr_doc)

In [ ]:
train_data[0]

tensor([    7,    29,  4031,  2997,   447,  3285,     4,   133,   444,    36,
         3745,     3,   838,     6, 24073,  1484,     4,    25, 13603,    71])

In [ ]:
train_data[1]

tensor([    7,  1781,    20,   179, 15130,   765,   632,    59,   490,  6997,
           27,     7,     6,     4,   480,  5051,    60,    15,     8,     9])

<bound method Field.build_vocab of <torchtext.data.field.Field object at 0x7f1ca121d400>>

In [ ]:
en_textfield.build_vocab(tr_dat, min_freq = 2)

In [ ]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (tr_dat, val_dat, test_dat),
    batch_size = BATCH_SIZE,
    device = device)

In [ ]:
bb

(tensor([[   9,   11, 3875,  ...,    4,    9,    9]]), tensor([2088628]))

In [ ]:
import tensorflow_datasets as tfds

In [ ]:
ds = tfds.load('glue', split='train', as_supervised=False, shuffle_files=False)

In [ ]:
for i in ds:
  print(i)

NameError: ignored

In [ ]:
aa.load()

AttributeError: ignored

In [ ]:
np.array([1,2,3]).shape()

TypeError: ignored

In [ ]:
ddd = aa(a = np.array([1,2,4,5]),b=np.array([1,2,4,5]))

In [ ]:
exec('b=ddd.a.sum()')

In [ ]:
len(np.array([1,2,3]))

3

In [ ]:
sample_dict.get('aa')

12

In [ ]:
sample_dict = {'aa': 12, 'bb':13}

In [ ]:
'list is list'.split(' ')

['list', 'is', 'list']

In [ ]:
dtype = torch.float
device = torch.device("cpu")

In [ ]:
#활용된 잡지식

#pytorch dataset 불러오기 
#https://pytorch.org/text/datasets.html

#자동으로 추가해야할 내용들
#To avoid clutter, we omit the implementation details including multi-head attention, residual connection, layer normalization and position-wise feed-forward as used in Transformer(-XL). The details are included in Appendix A.2 for reference

#데이터 로딩
#data loading
# https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html

#함수 오버라이딩 방법
#super(MyModel, self).__init__()

#einstein summation
#https://rockt.github.io/2018/04/30/einsum

#defining method with string
#https://stackoverflow.com/questions/11553721/using-a-string-variable-as-a-variable-name

#kwargs,args
#https://brunch.co.kr/@princox/180

In [ ]:
# Two-stream attention cell
#input : q_size, k_size, v_size, segment_index, attention_size, heads =8, **kwargs = {init_Qstream_q_LT_weight, init_Qstream_q_K_weight,init_Qstream_q_R_weight, init_Qstream_k_E_LT_weight, init_Qstream_k_R_LT_weight, init_Qstream_v_LT_weight,
#                                                                            init_Qstream_segment_same_weight, init_Qstream_segment_notsame_weight, init_Qstream_segment_bias, init_Qstream_output_weight,
#                                                                            init_Cstream_q_LT_weight, init_Cstream_q_K_weight,init_Cstream_q_R_weight, init_Cstream_k_E_LT_weight, init_Cstream_k_R_LT_weight, init_Cstream_v_LT_weight,
#                                                                            init_Cstream_segment_same_weight, init_Cstream_segment_notsame_weight, init_Cstream_segment_bias, init_Cstream_output_weight}
#

#operation : q_LT, k_LT, v_LT, segmentation, permutation(optional), masking, relative_position_encoding, segment_encoding, cal_content_attention, cal_query_attention, memorise

#properties : Qstream_q_LT_weight, Qstream_q_K_weight,Qstream_q_R_weight, Qstream_k_E_LT_weight, Qstream_k_R_LT_weight, Qstream_v_LT_weight,
#             Qstream_segment_same_weight, Qstream_segment_notsame_weight, Qstream_segment_bias, Qstream_output_weight,
#             Cstream_q_LT_weight, Cstream_q_K_weight,Cstream_q_R_weight, Cstream_k_E_LT_weight, Cstream_k_R_LT_weight, Cstream_v_LT_weight,
#             Cstream_segment_same_weight, Cstream_segment_notsame_weight, Cstream_segment_bias, Cstream_output_weight content_attention, query_attention, content_memory, query_memory

#output : Dq vector(s)



In [ ]:
class create_PLM_input(data_utils.Dataset):
  def __init__(self, data, target_csv, word_dict = False, sep_by_lines = False, segment_len = False, do_shuffle=True):
    super(create_PLM_input, self).__init__()
    
    

In [ ]:
class create_PLM_input_inmemory(data_utils.Dataset):


In [ ]:
# https://tutorials.pytorch.kr/beginner/data_loading_tutorial.html

perred


In [ ]:
a= torch.nn

In [ ]:
def shuffle()

def permutate()

def rel_position_encoding():

def masking():

In [ ]:
if not a:
  print(1)

In [ ]:
class Two_stream_attention(nn):
  def __init__(self, q_size, k_size, v_size, attention_size = False, heads =8, **kwargs):
    super(Two_stream_attention, self).__init__() 
      

    self.segment_index = segment_index

    #initialize params
    self.initialize_params(self, q_size, k_size, v_size, attention_size, heads)

  def initialize_params(self, q_size, k_size, v_size, attention_size, heads):
    self.Qstream_q_LT_weight = nn.Linear()
    self.Qstream_q_K_weight = 
    self.Qstream_q_R_weight = 
    
    self.Qstream_k_E_LT_weight = 
    self.Qstream_k_R_LT_weight = 
    self.Qstream_v_LT_weight = 
    
    self.Qstream_segment_same_weight = 
    self.Qstream_segment_notsame_weight = 
    self.Qstream_segment_bias = 
    
    self.Qstream_output_weight = 

    self.Cstream_q_LT_weight = 
    self.Cstream_q_K_weight = 
    self.Cstream_q_R_weight = 
    
    self.Cstream_k_E_LT_weight = 
    self.Cstream_k_R_LT_weight = 
    self.Cstream_v_LT_weight = 

    self.Cstream_segment_same_weight = 
    self.Cstream_segment_notsame_weight = 
    self.Cstream_segment_bias = 
    
    self.Cstream_output_weight =

  def set_params(self,**kwargs):
    for i,j in kwargs.items():
      setattr(self, i, j)
      explicitly_defined_vars.append(i)

  def forward(self, input, do_parm = False, rel_position = False):
    
    #record position
    if not rel_position:
      self.rel_position = rel_position_encoding(input)
    else:
      self.rel_position = rel_position

    #permutation
    if do_parm:
      self.permutate()

    #get position encoding

    #cal
  def backward(self,??): #클레스 정의하는 법 참조


SyntaxError: ignored

In [ ]:
_#http://nlp.seas.harvard.edu/2018/04/03/attention.html

In [ ]:
##Copyright 2013, Youngbin Jang, All rights reserved.

In [ ]:
#https://tutorials.pytorch.kr/beginner/nn_tutorial.html